# North Sea production visualization

Prepare data from UK and Norway databases for visualization in https://uber.github.io/kepler.gl/#/

Norway: https://www.norskpetroleum.no/en/facts/production/#north-sea

UK: https://data-ogauthority.opendata.arcgis.com/datasets/oga-field-production-pprs-wgs84

Lat.long : https://en.wikipedia.org/wiki/List_of_oil_and_gas_fields_of_the_North_Sea#United_Kingdom

Make video:https://www.macworld.co.uk/how-to/mac-software/how-record-screen-on-your-mac-3527168/

In [45]:
import os
import glob 
from datetime import datetime,date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

# Figures inline and set visualization style
%matplotlib inline
sns.set()
sns.set_style("whitegrid")

## Data preparation

In [46]:
df_UK = pd.read_csv('OGA_Field_Production_PPRS_WGS84.csv')
df_UK.fillna(0., inplace=True)

display(df_UK.head(5))

,OBJECTID,FIELDNAME,FIELDAREA,UNITNAME,UNITTYPCOD,UNITTYPDES,LOCATION,PERIODDATE,PERIODYRMN,PERIODMNTH,...,GCONDVOL,GCONDMBD,GCONDDEN,INJWATMASS,INJWATVOL,INJWATMBD,WATPRODMAS,WATPRODVOL,WATPRODMBD,ORGGRPNM
0,1,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-03-15T00:00:00.000Z,199403,3,...,0.0,0.0,0.0,74078.0,74078.0,15.030665,2192.0,2192.0,0.444764,CHEVRON CORPORATION
1,2,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-04-15T00:00:00.000Z,199404,4,...,0.0,0.0,0.0,217307.0,217307.0,45.562034,25516.0,25516.0,5.349855,CHEVRON CORPORATION
2,3,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-05-15T00:00:00.000Z,199405,5,...,0.0,0.0,0.0,334098.0,334098.0,67.789562,23496.0,23496.0,4.767414,CHEVRON CORPORATION
3,4,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-06-15T00:00:00.000Z,199406,6,...,0.0,0.0,0.0,297500.0,297500.0,62.375833,24628.0,24628.0,5.163671,CHEVRON CORPORATION
4,5,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-07-15T00:00:00.000Z,199407,7,...,0.0,0.0,0.0,321721.0,321721.0,65.278229,51591.0,51591.0,10.467980,CHEVRON CORPORATION


Extract all fields

In [47]:
#fields = df_UK['FIELDNAME'].unique()
#print(fields)

#df_UK_position = pd.DataFrame({'FIELDNAME':fields, 'LAT':np.zeros(len(fields)), 'LON':np.zeros(len(fields))})
#df_UK_position.to_csv('Fields_UK.csv')

Read file with fields and location (lat-long) and join to main dataframe

In [48]:
df_UK_position = pd.read_csv('Fields_UK.csv', index_col=0)
display(df_UK_position.head(5))

# fake data
#df_UK_position['LAT']=np.random.normal(58.0,1,len(df_UK_position))
#df_UK_position['LON']=np.random.normal(1.0,1,len(df_UK_position))
#display(df_UK_position.head(5))

df_UK = df_UK.merge(df_UK_position)

# get only data that have latitute
df_UK=df_UK[df_UK.LAT!=0]

display(df_UK.head(5))
print(df_UK.FIELDNAME.unique())

,FIELDNAME,LAT,LON
0,ALBA,58.05803,1.079633
1,AFFLECK,0.00000,0
2,ANDREW,58.05000,1.15
3,ALMA,0.00000,0
4,ALDER,0.00000,0


,OBJECTID,FIELDNAME,FIELDAREA,UNITNAME,UNITTYPCOD,UNITTYPDES,LOCATION,PERIODDATE,PERIODYRMN,PERIODMNTH,...,GCONDDEN,INJWATMASS,INJWATVOL,INJWATMBD,WATPRODMAS,WATPRODVOL,WATPRODMBD,ORGGRPNM,LAT,LON
0,1,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-03-15T00:00:00.000Z,199403,3,...,0.0,74078.0,74078.0,15.030665,2192.0,2192.0,0.444764,CHEVRON CORPORATION,58.05803,1.079633
1,2,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-04-15T00:00:00.000Z,199404,4,...,0.0,217307.0,217307.0,45.562034,25516.0,25516.0,5.349855,CHEVRON CORPORATION,58.05803,1.079633
2,3,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-05-15T00:00:00.000Z,199405,5,...,0.0,334098.0,334098.0,67.789562,23496.0,23496.0,4.767414,CHEVRON CORPORATION,58.05803,1.079633
3,4,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-06-15T00:00:00.000Z,199406,6,...,0.0,297500.0,297500.0,62.375833,24628.0,24628.0,5.163671,CHEVRON CORPORATION,58.05803,1.079633
4,5,ALBA,CNS,ALBA,OTL,Offshore Tanker Loader,Offshore,1994-07-15T00:00:00.000Z,199407,7,...,0.0,321721.0,321721.0,65.278229,51591.0,51591.0,10.467980,CHEVRON CORPORATION,58.05803,1.079633


['ALBA' 'ANDREW' 'AUK' 'BRAE-CENTRAL [Part of BRAE]' 'BLANE'
 'BRAE-NORTH [Part of BRAE]' 'BRAE-SOUTH [Part of BRAE]' 'BUCHAN' 'DRAKE'
 'ELGIN' 'CYRUS' 'FORTIES' 'EVEREST' 'FLEMING' 'FARRAGON' 'FULMAR'
 'FRANKLIN' 'GOLDEN EAGLE' 'HAWKINS' 'MILLER' 'SEYMOUR' 'HARDING'
 'KINNOULL' 'AMETHYST WEST' 'ANGLIA' 'LEMAN [PERENCO][pt. of LEMAN]'
 'CLAIR-PHASE 1 [Part of CLAIR]']


Choose only needec columns

In [49]:
df_UK = df_UK[['FIELDNAME', 'LAT', 'LON', 'PERIODDATE', 'OILPRODM3', 'DGASPROKSM', 'AGASPROMMS', 'WATPRODVOL']]
display(df_UK.head(5))
df_UK.rename(columns={'OILPRODM3': 'Oil Production Volumes', 
                      'DGASPROKSM': 'Dry Gas Production Volumes',
                      'AGASPROMMS': 'Associated Gas Production Volumes',
                      'WATPRODVOL': 'Water Production Volumes'}, inplace=True)
display(df_UK.head(5))

df_UK.to_csv('UK_Production_cleaned.csv', index=False)

,FIELDNAME,LAT,LON,PERIODDATE,OILPRODM3,DGASPROKSM,AGASPROMMS,WATPRODVOL
0,ALBA,58.05803,1.079633,1994-03-15T00:00:00.000Z,142134.0,0.0,6.691623,2192.0
1,ALBA,58.05803,1.079633,1994-04-15T00:00:00.000Z,166535.0,0.0,9.130105,25516.0
2,ALBA,58.05803,1.079633,1994-05-15T00:00:00.000Z,160681.0,0.0,9.311768,23496.0
3,ALBA,58.05803,1.079633,1994-06-15T00:00:00.000Z,198771.0,0.0,9.996499,24628.0
4,ALBA,58.05803,1.079633,1994-07-15T00:00:00.000Z,248810.0,0.0,12.571001,51591.0


,FIELDNAME,LAT,LON,PERIODDATE,Oil Production Volumes,Dry Gas Production Volumes,Associated Gas Production Volumes,Water Production Volumes
0,ALBA,58.05803,1.079633,1994-03-15T00:00:00.000Z,142134.0,0.0,6.691623,2192.0
1,ALBA,58.05803,1.079633,1994-04-15T00:00:00.000Z,166535.0,0.0,9.130105,25516.0
2,ALBA,58.05803,1.079633,1994-05-15T00:00:00.000Z,160681.0,0.0,9.311768,23496.0
3,ALBA,58.05803,1.079633,1994-06-15T00:00:00.000Z,198771.0,0.0,9.996499,24628.0
4,ALBA,58.05803,1.079633,1994-07-15T00:00:00.000Z,248810.0,0.0,12.571001,51591.0
